In [1]:
import pydriller
import pandas as pd
from tqdm import tqdm
import shutil

In [2]:
df = pd.read_csv('results.csv', encoding='latin-1')
df.head()

,name,isFork,commits,branches,defaultBranch,releases,contributors,license,watchers,stargazers,...,homepage,mainLanguage,totalIssues,openIssues,totalPullRequests,openPullRequests,lastCommit,lastCommitSHA,hasWiki,isArchived
0,knadh/ml2en,False,23,1,master,0,3.0,NaN,5,56,...,NaN,Python,4.0,1.0,4.0,0.0,2021-12-10T08:10:03,dd706167d52f8c9f556ee15869e00749c71df677,True,False
1,er10yi/magicude,False,98,2,main,12,1.0,NaN,12,423,...,NaN,Python,9.0,2.0,2.0,0.0,2022-01-10T03:28:29,b20f98939e928847f323d903f358221380a1ea74,True,False
2,keguoyu/minijvm,False,11,3,main,0,0.0,NaN,1,62,...,NaN,Python,NaN,NaN,NaN,NaN,2021-02-20T07:13:03,4da519cadead7164de37fb9b8913360dd45f9193,True,False
3,nordnet/next-api-v2-examples,False,35,2,master,0,10.0,MIT License,17,32,...,https://www.nordnet.se/externalapi/docs,Python,0.0,0.0,10.0,0.0,2022-03-10T11:18:36,73ed85404f10d879d8e5959a88a19dfad3b41827,False,False
4,aubreyrjones/parasol,False,70,1,master,0,1.0,BSD 2-Clause Simplified License,3,18,...,NaN,Python,0.0,0.0,0.0,0.0,2021-04-26T02:09:20,3110aec5b8824f10d324e35d02997b762f0436f4,True,False


In [3]:
df.shape

(152562, 25)

In [4]:
urls = df['name'].apply(lambda x: "https://github.com/" + x)
names = df['name'].apply(lambda x: x.split('/')[0])
urls[0], names[0]

('https://github.com/knadh/ml2en', 'knadh')

In [5]:
# urls = urls[:5]

In [6]:
from langdetect import detect

In [7]:
import os
if not os.path.exists('tmp_repos'):
    os.mkdir('tmp_repos')

In [8]:
def get_dir_size(path='.'):
    total = 0
    total_files = 0
    total_py_files = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
                total_files += 1
                if entry.name[-3:] == '.py':
                    total_py_files += 1
            elif entry.is_dir():
                a, b, c = get_dir_size(entry.path)
                total += a
                total_files += b
                total_py_files += c
    return total, total_files, total_py_files

In [9]:
assert len(urls) == len(names)

In [ ]:
MAX_SYMB_FILES_LEN = 5000
filters = ['numpy', 'pandas', 'sklearn', 'keras', 'torch', 'transformers', 'tensorflow',
           'scipy', 'statsmodels', 'flask', 'django', 'fastapi', 'gensim', 'nltk']

counter = 0
with open('good.txt', 'w') as f:
    pass
counter_no_fiters = 0
with open('good_no_filt.txt', 'w') as f:
    pass

In [11]:
urls = urls[108:]
names = names[108:]
assert len(urls) == len(names)

In [ ]:
pbar = tqdm(zip(urls, names), total=len(urls), position=0, leave=True)
for url, name in pbar:
    if not os.path.exists(f'tmp_repos/{name}'):
        os.mkdir(f'tmp_repos/{name}')
    repo = pydriller.Repository(url, clone_repo_to=f'tmp_repos/{name}', only_modifications_with_file_types='.py',
                                num_workers=6)
    try:
        commits = list(repo.traverse_commits())
    except:
        shutil.rmtree(f'tmp_repos/{name}')
        continue
    
    try:
        repo_size, total_files, total_py_files = get_dir_size(f'tmp_repos/{name}')
        if total_files > 60: # or repo_size * 10e-6 > 300: # or total_py_files / total_files < 0.2:
            shutil.rmtree(f'tmp_repos/{name}')
            continue
    except:
        continue
    
    for i, commit in enumerate(commits):
        try:
            if commit.files != 1:
                continue
            if commit.msg is None or commit.msg == '':
                continue
            if detect(commit.msg) != 'en':
                continue
            if commit.lines > 0 and commit.lines < 30:
                continue

            mod_files = commit.modified_files
            if len(mod_files) != 1 and mod_files[0].filename[-3:] != '.py':
                continue

            if mod_files[0].content_before is not None:
                code_before = mod_files[0].content_before.decode("utf-8") 
            else:
                continue

            if mod_files[0].content is not None:
                code_after = mod_files[0].content.decode("utf-8")
            else:
                continue

            if len(code_before) > MAX_SYMB_FILES_LEN or len(code_after) > MAX_SYMB_FILES_LEN:
                continue
            
            counter_no_fiters += 1
            with open('good_no_filt.txt', 'a') as f:
                f.write(str(url) + '\n')
            ANY_FILTER = False
            for filt in filters:
                if filt in code_before:
                    ANY_FILTER = True
                    break

            if ANY_FILTER:
                counter += 1
                with open('good.txt', 'a') as f:
                    f.write(str(url) + '\n')
            
            with open('log.txt', 'w') as f:
                f.write(str(counter_no_fiters))
                f.write('\n')
                f.write(str(counter))
        except:
            pass
    
    shutil.rmtree(f'tmp_repos/{name}')
    pbar.set_postfix({'num_good': counter, 'num_good_no_filter': counter_no_fiters})

  6%|▋         | 9646/152454 [8:48:55<136:56:06,  3.45s/it, num_good=409, num_good_no_filter=1819] 

In [1]:
counter

NameError: name 'counter' is not defined

In [ ]:
info